In [4]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import glob
import cv2

In [6]:
COLOR_LO = 0.7
COLOR_HI = 1.3

def get_file_name_dir(rootDir, ext):
 
    # Return file names with certain extension

    return glob.glob1(rootDir, "*." + ext)



def normalize_staining(img):
    """
    Adopted from "Classification of breast cancer histology images using Convolutional Neural Networks",
    Teresa Araújo , Guilherme Aresta, Eduardo Castro, José Rouco, Paulo Aguiar, Catarina Eloy, António Polónia,
    Aurélio Campilho. https://doi.org/10.1371/journal.pone.0177544

    Performs staining normalization.

    # Arguments
        img: Numpy image array.
    # Returns
        Normalized Numpy image array.
    """
    Io = 240
    beta = 0.15
    alpha = 1
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
    maxCRef = np.array([1.9705, 1.0308])

    h, w, c = img.shape
    img = img.reshape(h * w, c)
    OD = -np.log((img.astype("uint16") + 1) / Io)
    ODhat = OD[(OD >= beta).all(axis=1)]
    W, V = np.linalg.eig(np.cov(ODhat, rowvar=False))

    Vec = -V.T[:2][::-1].T  # desnecessario o sinal negativo
    That = np.dot(ODhat, Vec)
    phi = np.arctan2(That[:, 1], That[:, 0])
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100 - alpha)
    vMin = np.dot(Vec, np.array([np.cos(minPhi), np.sin(minPhi)]))
    vMax = np.dot(Vec, np.array([np.cos(maxPhi), np.sin(maxPhi)]))
    if vMin[0] > vMax[0]:
        HE = np.array([vMin, vMax])
    else:
        HE = np.array([vMax, vMin])

    HE = HE.T
    Y = OD.reshape(h * w, c).T

    C = np.linalg.lstsq(HE, Y)
    maxC = np.percentile(C[0], 99, axis=1)

    C = C[0] / maxC[:, None]
    C = C * maxCRef[:, None]
    Inorm = Io * np.exp(-np.dot(HERef, C))
    Inorm = Inorm.T.reshape(h, w, c).clip(0, 255).astype("uint8")

    return Inorm


def hematoxylin_eosin_aug(img, low=0.7, high=1.3, seed=None):
    """
    "Quantification of histochemical staining by color deconvolution"
    Arnout C. Ruifrok, Ph.D. and Dennis A. Johnston, Ph.D.
    http://www.math-info.univ-paris5.fr/~lomn/Data/2017/Color/Quantification_of_histochemical_staining.pdf

    Performs random hematoxylin-eosin augmentation

    # Arguments
        img: Numpy image array.
        low: Low boundary for augmentation multiplier
        high: High boundary for augmentation multiplier
    # Returns
        Augmented Numpy image array.
    """
    D = np.array([[1.88, -0.07, -0.60],
                  [-1.02, 1.13, -0.48],
                  [-0.55, -0.13, 1.57]])
    M = np.array([[0.65, 0.70, 0.29],
                  [0.07, 0.99, 0.11],
                  [0.27, 0.57, 0.78]])
    Io = 240

    h, w, c = img.shape
    OD = -np.log10((img.astype("uint16") + 1) / Io)
    C = np.dot(D, OD.reshape(h * w, c).T).T
    r = np.ones(3)
    r[:2] = np.random.RandomState(seed).uniform(low=low, high=high, size=2)
    img_aug = np.dot(C, M) * r

    img_aug = Io * np.exp(-img_aug * np.log(10)) - 1
    img_aug = img_aug.reshape(h, w, c).clip(0, 255).astype("uint8")
    return img_aug

def process_image(image_file):
    
#     img = cv2.imread(image_file)
    img = cv2.cvtColor(image_file, cv2.COLOR_BGR2RGB)
    img_norm = normalize_staining(img)
    img_aug = hematoxylin_eosin_aug(img_norm, low=COLOR_LO, high=COLOR_HI)

    return img_aug

In [7]:
input_path = 'data/Benign/'
input_extension = 'tif'

inputs_files = sorted(get_file_name_dir(input_path, input_extension))

cont = 1

for file_name in inputs_files:
    print('Preprocessing: ' + file_name + ': ' + str(cont))
    cont += 1
 
    img = input_path + file_name
    im_in = cv2.imread(img)
    image = process_image(im_in)

    cv2.imwrite(input_path +file_name , image)

Preprocessing: b001.tif: 1
Preprocessing: b002.tif: 2
Preprocessing: b003.tif: 3
Preprocessing: b004.tif: 4
Preprocessing: b005.tif: 5
Preprocessing: b006.tif: 6
Preprocessing: b007.tif: 7
Preprocessing: b008.tif: 8
Preprocessing: b009.tif: 9
Preprocessing: b010.tif: 10
Preprocessing: b011.tif: 11
Preprocessing: b012.tif: 12
Preprocessing: b013.tif: 13
Preprocessing: b014.tif: 14
Preprocessing: b015.tif: 15
Preprocessing: b016.tif: 16
Preprocessing: b017.tif: 17
Preprocessing: b018.tif: 18
Preprocessing: b019.tif: 19
Preprocessing: b020.tif: 20
Preprocessing: b021.tif: 21
Preprocessing: b022.tif: 22
Preprocessing: b023.tif: 23
Preprocessing: b024.tif: 24
Preprocessing: b025.tif: 25
Preprocessing: b026.tif: 26
Preprocessing: b027.tif: 27
Preprocessing: b028.tif: 28
Preprocessing: b029.tif: 29
Preprocessing: b030.tif: 30
Preprocessing: b031.tif: 31
Preprocessing: b032.tif: 32
Preprocessing: b033.tif: 33
Preprocessing: b034.tif: 34
Preprocessing: b035.tif: 35
Preprocessing: b036.tif: 36
P